In [0]:
old_data = [
    (101, 'Naina', 'Noida'),
    (102, 'Anu', 'Pune')
]
columns = ["Customer_id", "Name", "City"]
old_df = spark.createDataFrame(old_data, columns)
display(old_df)


In [0]:
new_data = [
    (101, 'Naina', 'Noida'),
    (102, 'Anu', 'Noida'),
    (103, 'Akshay', 'Noida')
]

new_df = spark.createDataFrame(new_data, columns)
display(new_df)

## SCD Type-1: oVerwrite(No History)

In [0]:
#Save the old data as Delta Table
from delta.tables import DeltaTable

# Write as a managed table
old_df.write.format("delta").mode("overwrite").saveAsTable("dim_customers_type1")

# Read it as DeltaTable
dim_customers = DeltaTable.forName(spark, "dim_customers_type1")

In [0]:
#Merge for SCD Type 1
dim_customers.alias("target").merge(
new_df.alias("source"),
"target.customer_id = source.customer_id"
).whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

## SCD TYPE-2: Keep History

In [0]:
#  SCD TYPE-2: Keep History
from pyspark.sql.functions import lit, current_date

df_new_scd2 = new_df.withColumn("is_active", lit(True)) \
.withColumn("start_date", current_date()) \
.withColumn("end_date", lit(None).cast("date"))

In [0]:
#adding audit filed to old result
old_df_scd2 = old_df.withColumn("is_active", lit(True)) \
.withColumn("start_date", lit("2024-01-01").cast("date")) \
.withColumn("end_date", lit(None).cast("date"))

In [0]:
#adding audit filed to new
new_df_scd2 = new_df.withColumn("is_active", lit(True)) \
.withColumn("start_date", lit("2024-01-01").cast("date")) \
.withColumn("end_date", lit(None).cast("date"))

In [0]:
display(new_df_scd2)

In [0]:
# Expire changes records
from pyspark.sql.functions import lit, current_date

df_expired = (
    old_df_scd2.alias("old")
    .join(new_df.alias("new"), "customer_id")
    .filter("old.city != new.city")                # compare old vs new
    .select("old.customer_id", "old.name", "old.city")
    .withColumn("is_active", lit(False))           # mark old record inactive
    .withColumn("start_date", lit("2024-01-01").cast("date"))  # start date cast to date
    .withColumn("end_date", current_date())        # end date = today
)

display(df_expired)


In [0]:
#unchanged records
df_unchanged = old_df_scd2.alias("old").join(new_df.alias("new"), "customer_id") \
.filter("old.city == new.city")
display(df_unchanged)

In [0]:
#New record entry

df_new_only = new_df.alias("new").join(old_df.alias("old"), "customer_id", "leftanti") \
.withColumn("is_active", lit(True)) \
.withColumn("start_date", current_date()) \
.withColumn("end_date", lit(None).cast("date"))

display(df_new_only)

## Databricks SQL Implementation

In [0]:
%sql
-- STEP 1: CREATE DIMENSION TABLE
CREATE OR REPLACE TABLE dim_customers(
  customer_id INT,
  name STRING,
  city STRING
) USING DELTA;

INSERT INTO dim_customers VALUES
(101, 'Ankur', 'Pune'),
(102, 'Shiv', 'Banglore')

In [0]:
%sql
-- STEP 2: CREATE STAGING VIEW
CREATE OR REPLACE TEMP VIEW staging_customers AS SELECT * FROM VALUES
(101, 'Ankur', 'Pune'),
(102, 'Shiv', 'Banglore'),
(103, 'Ravi', 'Delhi')
AS t(customer_id, name, city);

In [0]:
%sql
select * from staging_customers

##SCD TYPE 1

In [0]:
%sql
--MERGE INTO CMND IS USE TO MAKE SURE DUPLICATE VALUES ARE NOT THERE

MERGE INTO dim_customers AS target
USING staging_customers AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
SELECT * FROM dim_customers

##SCD TYPE 2

In [0]:
 %sql
 -- STEP 1 CREATE AUDIT field
 CREATE OR REPLACE TABLE dim_customers(
  customer_id INT,
  name STRING,
  city STRING,
  is_active BOOLEAN,
  start_date DATE,
  end_date DATE
 ) USING DELTA;

In [0]:
%sql
 INSERT INTO dim_customers VALUES
 (101, 'Ankur', 'Pune', true, DATE('2024-01-01'), NULL),
 (102, 'stuti', 'Mumbai', true, DATE('2024-01-01'), NULL);

In [0]:
%sql
select * from dim_customers

In [0]:
%sql
-- STEP 2: EXPIRE OLD
UPDATE dim_customers
SET is_active = false,
end_date = current_date()
WHERE customer_id IN(
  SELECT s.customer_id FROM staging_customers s JOIN dim_customers d
  ON s.customer_id = d.customer_id
  WHERE s.city != d.city AND d.is_active = true
);

In [0]:
%sql
select * from dim_customers

In [0]:
%sql
-- STEP3: INSERT NEW
INSERT INTO dim_customers
SELECT s.customer_id , s.name, s.city,
true, current_date(), null
FROM staging_customers s
LEFT JOIN dim_customers d
ON s.customer_id = d.customer_id AND d.is_active = true
WHERE d.city IS NULL OR s.city != d.city;

In [0]:
%sql
select * from dim_customers